In [32]:

#pip install pytesseract
#pip install PyPDF2
#pip install pdfplumber
#pip install ocrmypdf
#pip install PyMuPDF

In [ ]:
#para trabajar con ocrmypdf se debe intalar adicionalmente: Ghostscript y Tesseract OCR


In [14]:
import os
import pytesseract
from PIL import Image
import PyPDF2
#import ocrmypdf
import cv2
import pytesseract
import pandas as pd

### pasar pdf a imagen y recortarla

In [19]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import os

# Ruta del archivo PDF de entrada
pdf_path = "C:\\Users\\pc\\OneDrive\\Escritorio\\OC_pdf\\SEGURO DE VIDA Reina Quintana Sergio Ivan.pdf"

# Directorio donde se guardarán las imágenes
output_directory = "C:\\Users\\pc\\OneDrive\\Escritorio\\OC_pdf\\Nueva carpeta\\"

# Número de página que deseas convertir (página 0 es la primera página)
page_number = 0

# Abre el archivo PDF
pdf_document = fitz.open(pdf_path)

# Asegúrate de que la carpeta de salida exista, o créala si no existe
os.makedirs(output_directory, exist_ok=True)

# Selecciona la página
page = pdf_document.load_page(page_number)

# Renderiza la página como una imagen (puedes ajustar la resolución)
pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

# Construye la ruta completa para guardar la imagen
image_path = os.path.join(output_directory, f"pagina_{page_number + 1}.png")

# Guarda la imagen como un archivo PNG en la ruta especificada
pix.save(image_path)

# Cierra el documento PDF
pdf_document.close()

print(f"Se ha convertido la página {page_number + 1} en una imagen y se ha guardado en {image_path}.")

# Carga la imagen convertida
image = cv2.imread(image_path)

# Convierte la imagen a escala de grises
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Aplica la detección de bordes (Canny)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Aplica la transformada de Hough para detectar líneas
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

# Encuentra los puntos extremos de las líneas detectadas y recorta la imagen
if lines is not None:
    # Inicializa una máscara en escala de grises del mismo tamaño que la original
    mask = np.zeros(gray.shape, dtype=np.uint8)

    for line in lines:
        x1, y1, x2, y2 = line[0]
        
        # Dibuja la línea en la máscara (blanco)
        cv2.line(mask, (x1, y1), (x2, y2), 255, 2)

    # Encuentra los contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Recorta la región del rectángulo en la imagen original
        imagen_recortada = image[y:y + h, x:x + w]

        # Redimensiona la imagen recortada al tamaño deseado (por ejemplo, 1629x1236 píxeles)
        nuevo_ancho = 1629
        nuevo_alto = 1236
        imagen_recortada = cv2.resize(imagen_recortada, (nuevo_ancho, nuevo_alto))

        # Aplica un filtro de mejora de nitidez a la imagen
        imagen_mejorada = cv2.filter2D(imagen_recortada, -1, np.array([[-1, -1, -1],
                                                                     [-1,  9, -1],
                                                                     [-1, -1, -1]]))

        # Ruta y nombre de la imagen recortada y mejorada
        nombre_imagen_recortada = os.path.join(output_directory, f"pagina_{page_number + 1}.png")
        nombre_imagen_mejorada = os.path.join(output_directory, f"pagina_{page_number + 1}.png")

        # Guarda la imagen recortada y la imagen mejorada en las rutas especificadas
        #cv2.imwrite(nombre_imagen_recortada, imagen_recortada)
        cv2.imwrite(nombre_imagen_mejorada, imagen_mejorada)

        #cv2.imshow("Imagen Recortada", imagen_recortada)
        cv2.imshow("Imagen Mejorada", imagen_mejorada)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No se encontraron contornos en la máscara")
else:
    print("No se encontraron líneas en la imagen")



Se ha convertido la página 1 en una imagen y se ha guardado en C:\Users\pc\OneDrive\Escritorio\OC_pdf\Nueva carpeta\pagina_1.png.


In [12]:
import cv2

# Carga la imagen
image = cv2.imread("C:\\Users\\pc\\OneDrive\\Escritorio\\OC_pdf\\Nueva carpeta\\pagina_1.png")

# Crea una ventana
cv2.namedWindow('Imagen', cv2.WINDOW_NORMAL)  # Puedes ajustar el tamaño de la ventana si lo deseas

# Muestra la imagen en la ventana
cv2.imshow('Imagen', image)

# Función para manejar los clics del mouse
def obtener_coordenadas(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'Coordenadas: ({x}, {y})')

# Asigna la función de manejo de clics al evento del mouse
cv2.setMouseCallback('Imagen', obtener_coordenadas)

# Espera a que el usuario haga clic en la imagen
cv2.waitKey(0)

# Cierra la ventana
cv2.destroyAllWindows()

Coordenadas: (748, 119)
Coordenadas: (1116, 246)


In [20]:
import cv2
import pytesseract

# Carga la imagen
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\pc\Documents\tesseract.exe"  # Reemplaza esta ruta con la ubicación real de Tesseract en tu sistema

image = cv2.imread("C:\\Users\\pc\\OneDrive\\Escritorio\\OC_pdf\\Nueva carpeta\\pagina_1.png")

# Define las coordenadas de las regiones de la imagen que deseas extraer
regiones = [
    {"left_x": 5, "top_y": 126, "right_x": 747, "bottom_y": 238},
    {"left_x": 749, "top_y": 139, "right_x": 1114, "bottom_y": 248},
    {"left_x": 1115, "top_y": 129, "right_x": 1505, "bottom_y": 239},
    {"left_x": 1507, "top_y": 138, "right_x": 1623, "bottom_y": 245},
    {"left_x": 5, "top_y": 226, "right_x": 747, "bottom_y": 338},
    {"left_x": 749, "top_y": 239, "right_x": 1115, "bottom_y": 349},
    {"left_x": 1116, "top_y": 239, "right_x": 1504, "bottom_y": 351},
    {"left_x": 1507, "top_y": 243, "right_x": 1623, "bottom_y": 353},
]

# Lista para almacenar el texto extraído de cada región
beneficiarios = []
Beneficiarios =[]
for region in regiones:
    left_x = region["left_x"]
    top_y = region["top_y"]
    right_x = region["right_x"]
    bottom_y = region["bottom_y"]

    # Recorta la región de la imagen según las coordenadas
    cropped_image = image[top_y:bottom_y, left_x:right_x]

    # Convierte la imagen recortada a escala de grises
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # Extrae el texto de la imagen recortada
    extracted_text = pytesseract.image_to_string(gray_image, lang='spa', config='--psm 6')

    # Recorta la región de la imagen según las coordenadas
    cropped_image = image[top_y:bottom_y, left_x:right_x]

    # Convierte la imagen recortada a escala de grises
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # Extrae el texto de la imagen recortada
    extracted_text = pytesseract.image_to_string(gray_image)

    # Imprime el texto extraído o guárdalo en un archivo, según tus necesidades
    #print(f"Texto extraído de región {idx + 1}:")

    Beneficiarios.append(extracted_text)
    
print(Beneficiarios)



['Angelica Vannessa Ramirez Rubio\n\n', '11 | 12 | 92\n', '1072667399\n', 'ESPOS@\n', 'German Reina Bojaca\n', '11 | 03 |59\n', '2994376\n', 'SUEGR@\nO PADRE\n']


In [21]:
if (len(Beneficiarios)%4==0):
    
    chunks = [Beneficiarios[i:i+4] for i in range(0, len(Beneficiarios), 4)]
    df = pd.DataFrame(chunks, columns=['Nombre', 'Fecha_Nacimiento', 'Documento_id', 'Parentesco'])
else:
    print("dimención incompleta")
    
df

,Nombre,Fecha_Nacimiento,Documento_id,Parentesco
0,Angelica Vannessa Ramirez Rubio\n\n,11 | 12 | 92\n,1072667399\n,ESPOS@\n
1,German Reina Bojaca\n,11 | 03 |59\n,2994376\n,SUEGR@\nO PADRE\n


In [22]:
from datetime import datetime

def convertir_fecha(fecha_str):
    # Divide la cadena en función del carácter "|"
    partes = fecha_str.split("|")

    # Elimina los espacios en blanco y el salto de línea de cada parte y convierte a enteros
    dia = int(partes[0].strip())
    mes = int(partes[1].strip())
    ano = int(partes[2].strip())

    # Ajusta el valor del año para que sea "1992" en lugar de "92"
    if ano >= 0 and ano <= 99:
        ano += 1900

    # Crea un objeto de fecha
    return datetime(year=ano, month=mes, day=dia)


df['Fecha_Nacimiento'] = [convertir_fecha(fecha) for fecha in df['Fecha_Nacimiento']]
df



,Nombre,Fecha_Nacimiento,Documento_id,Parentesco
0,Angelica Vannessa Ramirez Rubio\n\n,1992-12-11,1072667399\n,ESPOS@\n
1,German Reina Bojaca\n,1959-03-11,2994376\n,SUEGR@\nO PADRE\n


In [23]:
import re

# Lista de cadenas con el patrón " | "
lista = df['Nombre']

# Define el patrón a eliminar
patron = r'\n'  # El espacio y las barras verticales deben escaparse con "\"

# Utiliza re.sub() para eliminar el patrón en cada cadena de la lista
df['Nombre']= [re.sub(patron, '', cadena) for cadena in df['Nombre']]
df['Documento_id']= [re.sub(patron, '', cadena) for cadena in df['Documento_id']]
df['Parentesco']= [re.sub(patron, '', cadena) for cadena in df['Parentesco']]
# Imprime la lista resultante
df


,Nombre,Fecha_Nacimiento,Documento_id,Parentesco
0,Angelica Vannessa Ramirez Rubio,1992-12-11,1072667399,ESPOS@
1,German Reina Bojaca,1959-03-11,2994376,SUEGR@O PADRE
